In [12]:
import requests
import re
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import deepl
import time
import random
import os
from datetime import datetime

In [13]:
def backtranslate_dialogue(text, translator):
    if not text:
        return text
        
    lines = text.split('\n')
    result = []
    
    for line in lines:
        speaker = line[:11]
        content = line[11:]
        
        try:
            # DeepL의 공식 언어 코드 사용
            translated = translator.translate_text(
                content, 
                source_lang='KO',  # 소스 언어는 'KO'
                target_lang='EN-US'  # 타겟 언어는 'EN-US'
            ).text
            
            back_translated = translator.translate_text(
                translated, 
                source_lang='EN',  # 영어에서 다시 번역할 때는 'EN'
                target_lang='KO'   # 한국어로 번역할 때는 'KO'
            ).text
            
            result.append(speaker + back_translated)
            
        except Exception as e:
            print(f"번역 오류: {str(e)}")
            result.append(line)
    
    return '\n'.join(result)

def process_backtranslations(df, save_path):
    translator = deepl.Translator('3e69e331-9c9f-4cf5-811c-8fcd910e36da')
    
    print(f"번역 시작: {datetime.now().strftime('%H:%M:%S')}")
    
    result_df = df.copy()
    result_df.index += len(df)
    result_df['fname'] = [f'train_{i}' for i in result_df.index]
    
    try:
        for idx in range(len(df)):
            result_df.loc[idx + len(df), 'dialogue'] = backtranslate_dialogue(
                df.loc[idx, 'dialogue'], 
                translator
            )
            
            if (idx + 1) % 10 == 0:
                print(f"진행: {idx + 1}/{len(df)}")
                result_df.to_csv(save_path, index=False)
                
    except KeyboardInterrupt:
        print("\n저장 중...")
    finally:
        result_df.to_csv(save_path, index=False)
        print(f"완료: {datetime.now().strftime('%H:%M:%S')}")
        print(f"저장 위치: {save_path}")

In [14]:
# Read the original CSV file
df = pd.read_csv("/root/NLP/data/train.csv")

In [16]:
df_1 = df.tail(-3001).head(4000)

In [17]:
#인덱스 리셋
df_1.reset_index(drop=True, inplace=True)

# fname 열을 'train_0', 'train_1', ... 으로 변환
df_1['fname'] = [f"train_{i}" for i in range(len(df_1))]

In [19]:
# 실행
save_dir = '/root/NLP/data/dt_train.csv'
os.makedirs(save_dir, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(save_dir, f'backtranslated_{timestamp}.csv')

df = pd.read_csv('/root/NLP/data/train.csv')
process_backtranslations(df_1, save_path)

번역 시작: 15:03:14


진행: 10/4000
진행: 20/4000
진행: 30/4000
진행: 40/4000
진행: 50/4000
진행: 60/4000
진행: 70/4000
진행: 80/4000
진행: 90/4000
진행: 100/4000
진행: 110/4000
진행: 120/4000
진행: 130/4000
진행: 140/4000
진행: 150/4000
진행: 160/4000
진행: 170/4000
진행: 180/4000
진행: 190/4000
진행: 200/4000
진행: 210/4000
진행: 220/4000
진행: 230/4000
진행: 240/4000
진행: 250/4000
진행: 260/4000
진행: 270/4000
진행: 280/4000
진행: 290/4000
진행: 300/4000
진행: 310/4000
진행: 320/4000
진행: 330/4000
진행: 340/4000
진행: 350/4000
진행: 360/4000
진행: 370/4000
진행: 380/4000
진행: 390/4000
진행: 400/4000
진행: 410/4000
진행: 420/4000
진행: 430/4000
진행: 440/4000
진행: 450/4000
진행: 460/4000
진행: 470/4000
진행: 480/4000
진행: 490/4000
진행: 500/4000
진행: 510/4000
진행: 520/4000
진행: 530/4000
진행: 540/4000
진행: 550/4000
진행: 560/4000
진행: 570/4000
진행: 580/4000
진행: 590/4000
진행: 600/4000
진행: 610/4000
진행: 620/4000
진행: 630/4000
진행: 640/4000
진행: 650/4000
진행: 660/4000
진행: 670/4000
진행: 680/4000
진행: 690/4000
진행: 700/4000
진행: 710/4000
진행: 720/4000
진행: 730/4000
진행: 740/4000
진행: 750/4000
진행: 760/4000
진행: 770/4000
진행: 780/

In [7]:
dt_df = pd.read_csv("/root/NLP/data/dt_train.csv/backtranslated_20241127_082909.csv")

In [8]:
all_df = pd.concat([df, dt_df],axis=0)

In [9]:
all_df.head()

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스


In [12]:
co_df = pd.read_csv("/root/NLP/data/dt_train.csv/(co)backtranslated_20241127_091942.csv")

In [14]:
co_df.head()

,fname,dialogue,summary,topic
0,train_1000,#Person1#: 거주하시는 도시의 날씨는 어떤가요?\n#Person2#: 여름에...,"#Person2#의 도시에서는 여름에는 덥고 비가 오며, 겨울에는 더 건조하다. 하...",도시 날씨
1,train_1001,"#Person1#: 안녕하세요, 무엇을 도와드릴까요?\n#Person2#: 미국식 ...","#Person1#은 #Person2#가 햇볕에 익힌 계란과 브렉퍼스트 주스, 진한 ...",음식 주문
2,train_1002,"#Person1#: 분명 후회하지 않으실 거라고 확신하며, 그보다 더 좋은 사람은 ...",#Person1#은 #Person2#가 그 여자를 선택하지 않으면 후회할 것이라고 ...,후회
3,train_1003,"#Person1#: 실례합니다만, 저 좀 도와주시겠어요?\n#Person2#: 예....",#Person1#은 #Person2#에게 꽃 전시회에 대해 물어본다. #Person...,꽃 전시회
4,train_1004,#Person1#: 분실물 센터에 있습니다. 어떻게 도와드릴까요?\n#Person2...,#Person2#의 가방이 어제 분실되었다. #Person2#가 제공한 정보를 바탕...,분실물 찾기


In [15]:
all_co_df = pd.concat([all_df, co_df])

In [16]:
#인덱스 리셋
all_co_df.reset_index(drop=True, inplace=True)

# fname 열을 'train_0', 'train_1', ... 으로 변환
all_co_df['fname'] = [f"train_{i}" for i in range(len(all_co_df))]

# 중복 제거 데이터 저장 (원하는 경로로 변경)
output_path = "/root/NLP/data/dt1_train.csv"
all_co_df.to_csv(output_path, index=False)

print("CSV 파일이 성공적으로 저장되었습니다.")

CSV 파일이 성공적으로 저장되었습니다.
